In [1]:
import os
import datetime
import glob
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds 
import seaborn as sns
import scipy.spatial
import sklearn.manifold
from tensorflow.keras.utils import to_categorical
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from PIL import Image

In [2]:
network = tf.keras.applications.resnet50.ResNet50(weights = "imagenet")

In [3]:
dataset_name = 'oxford_iiit_pet'
dataset, info = tfds.load(name=dataset_name, split='train', with_info=True)
test_dataset, test_info = tfds.load(name=dataset_name, split='test', with_info=True)

In [4]:
feature_extractor = tf.keras.Model(
    inputs = network.input,
    outputs = network.layers[-2].output
)

In [5]:
def preprocess_and_extract_embedding(example):
    image = example['image']
    resized_image = tf.image.resize(image, (224, 224))
    preprocessed_image = tf.keras.applications.resnet50.preprocess_input(resized_image)
    embedding = feature_extractor(preprocessed_image[np.newaxis, ...])
    return tf.squeeze(embedding)
    

In [6]:
train_embeddings = [preprocess_and_extract_embedding(example) for example in dataset]

In [7]:
train_labels = [example['label'] for example in dataset]
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, num_classes=37)

In [8]:
test_embeddings = [preprocess_and_extract_embedding(example) for example in test_dataset]
test_labels = [example['label'] for example in test_dataset]
t_labels = tf.keras.utils.to_categorical(test_labels, num_classes=37)

In [9]:
print(len(train_embeddings))

3680


In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=37, activation='softmax', input_shape=(2048,))
    # Replace '1' with the number of units for multiclass classification
    # Use 'softmax' activation for multiclass classification
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 37)                75813     
                                                                 
Total params: 75813 (296.14 KB)
Trainable params: 75813 (296.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.trainable = True

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [84]:
model.fit(np.array(train_embeddings), train_labels_one_hot, epochs=10, batch_size=3680)

Epoch 1/10
1/1 [==============================] - 0s 65ms/step - loss: 0.0171 - accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 17ms/step - loss: 0.0170 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 14ms/step - loss: 0.0170 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 15ms/step - loss: 0.0169 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 15ms/step - loss: 0.0168 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 16ms/step - loss: 0.0167 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 13ms/step - loss: 0.0166 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 15ms/step - loss: 0.0165 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 18ms/step - loss: 0.0164 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 14ms/step - loss: 0.0163 - accuracy: 1.0000


In [85]:
predict_labels = model.predict(np.array(test_embeddings))
one_hot_vectors = np.zeros_like(predict_labels)
one_hot_vectors[np.arange(len(predict_labels)), np.argmax(predict_labels, axis=1)] = 1

115/115 [==============================] - 0s 849us/step


In [86]:
test_accuracy = accuracy_score(t_labels, one_hot_vectors)
print(f'Accuracy on the test set: {test_accuracy * 100:.2f}%')

Accuracy on the test set: 87.93%
